# Demand Forecasting Demo — ARIMA/Prophet vs LSTM/GRU
Author: *Chibuogwu Onyemaechi*

**Goal:** simple, interview-ready time-series forecasting demo (water/utility style) that compares a classical model (ARIMA/Prophet) with a small deep learning model (LSTM/GRU).

Dataset: classic **AirPassengers** monthly series (embedded; no internet needed).
You can swap in electricity/water demand easily by changing the loader.


## 1) Setup

In [3]:

# If running on Colab, uncomment to install what you need
# !pip -q install numpy pandas matplotlib scikit-learn statsmodels pmdarima prophet tensorflow

import warnings, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

warnings.filterwarnings("ignore")
plt.rcParams["figure.dpi"] = 140

OUT = Path("outputs"); OUT.mkdir(exist_ok=True)
print("Saving figures to:", OUT.resolve())


Saving figures to: C:\Users\ochib\Downloads\demand_forecasting\outputs


## 2) Load a simple monthly series (AirPassengers)

In [ ]:

# Embedded AirPassengers (1949-1960), monthly
values = [112, 118, 132, 129, 121, 135, 148, 148, 136, 119, 104, 118, 115, 126, 141, 135, 125, 149, 170, 170, 158, 133, 114, 140, 145, 150, 178, 163, 172, 178, 199, 199, 184, 162, 146, 166, 171, 180, 193, 181, 183, 218, 230, 242, 209, 191, 172, 194, 196, 196, 236, 235, 229, 243, 264, 272, 237, 211, 180, 201, 204, 188, 235, 227, 234, 264, 302, 293, 259, 229, 203, 229, 242, 233, 267, 269, 270, 315, 364, 347, 312, 274, 237, 278, 284, 277, 317, 313, 318, 374, 413, 405, 355, 306, 271, 306, 315, 301, 356, 348, 355, 422, 465, 467, 404, 347, 305, 336, 340, 318, 362, 348, 363, 435, 491, 505, 404, 359, 310, 337, 360, 342, 406, 396, 420, 472, 548, 559, 463, 407, 362, 405, 417, 391, 419, 461, 472, 535, 622, 606, 508, 461, 390, 432]

# Build a monthly DateTimeIndex
dates = pd.date_range(start="1949-01-01", periods=len(values), freq="MS")
ts = pd.Series(values, index=dates, name="passengers")

ts.head(), ts.tail(), ts.plot(title="AirPassengers (monthly)") ; plt.show()


## 3) Train/Validation split

In [ ]:

# Use the last 24 months as validation
HORIZON = 12         # forecast horizon we'll evaluate on
VAL_LEN = 24

train = ts.iloc[:-VAL_LEN]
val   = ts.iloc[-VAL_LEN:]

ax = train.plot(label="train")
val.plot(ax=ax, label="val")
plt.axvline(val.index[0], color="k", ls="--", alpha=0.6)
plt.legend(); plt.title("Train / Validation split"); plt.tight_layout()
plt.savefig(OUT/"split.png", dpi=200); plt.show()

print("Train:", train.index.min(), "→", train.index.max(), "| len:", len(train))
print("Val:  ", val.index.min(),   "→", val.index.max(),   "| len:", len(val))


## 4) Classical: ARIMA (auto_arima)

In [ ]:

try:
    import pmdarima as pm
    model = pm.auto_arima(train, seasonal=True, m=12, stepwise=True, trace=False, suppress_warnings=True)
    print(model.summary())
    # Forecast on validation span
    fc = model.predict(n_periods=len(val))
    fc = pd.Series(fc, index=val.index, name="ARIMA_forecast")

    ax = train.plot(label="train", figsize=(8,4))
    val.plot(ax=ax, label="val")
    fc.plot(ax=ax, label="ARIMA fc", linestyle="--")
    plt.legend(); plt.title("ARIMA forecast vs validation"); plt.tight_layout()
    plt.savefig(OUT/"arima_val.png", dpi=200); plt.show()

    def metrics(y_true, y_pred):
        mae  = np.mean(np.abs(y_true - y_pred))
        rmse = np.sqrt(np.mean((y_true - y_pred)**2))
        mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100
        return mae, rmse, mape

    mae, rmse, mape = metrics(val.values, fc.values)
    print(f"ARIMA val → MAE={mae:.2f} | RMSE={rmse:.2f} | MAPE={mape:.2f}%")

except Exception as e:
    print("⚠️ ARIMA unavailable:", e)
    fc = None


## 5) Prophet (optional)

In [ ]:

try:
    from prophet import Prophet
    df = train.reset_index().rename(columns={"index":"ds","passengers":"y"})
    m = Prophet(seasonality_mode="multiplicative", yearly_seasonality=True)
    m.fit(df)
    future = pd.DataFrame({"ds": val.index})
    fc_prophet = m.predict(future)[["ds","yhat"]].set_index("ds")["yhat"]
    fc_prophet.name = "Prophet_forecast"

    ax = train.plot(label="train", figsize=(8,4))
    val.plot(ax=ax, label="val")
    fc_prophet.plot(ax=ax, label="Prophet fc", linestyle="--")
    plt.legend(); plt.title("Prophet forecast vs validation"); plt.tight_layout()
    plt.savefig(OUT/"prophet_val.png", dpi=200); plt.show()

    def metrics(y_true, y_pred):
        mae  = np.mean(np.abs(y_true - y_pred))
        rmse = np.sqrt(np.mean((y_true - y_pred)**2))
        mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100
        return mae, rmse, mape

    mae, rmse, mape = metrics(val.values, fc_prophet.values)
    print(f"Prophet val → MAE={mae:.2f} | RMSE={rmse:.2f} | MAPE={mape:.2f}%")

except Exception as e:
    print("⚠️ Prophet unavailable:", e)
    fc_prophet = None


## 6) Deep Learning: small LSTM (TensorFlow)

In [ ]:

def make_supervised(series, window=12):
    X, y = [], []
    vals = series.values.astype(np.float32)
    for i in range(len(vals) - window):
        X.append(vals[i:i+window])
        y.append(vals[i+window])
    return np.array(X), np.array(y)

try:
    import tensorflow as tf
    tf.get_logger().setLevel("ERROR")
    WINDOW = 12

    X_tr, y_tr = make_supervised(train, window=WINDOW)
    # Fit on train; validate on the next VAL_LEN points
    # For fair comparison, only generate predictions on the val tail
    # Build a simple LSTM
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(WINDOW,1)),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")
    model.summary()

    hist = model.fit(X_tr.reshape(-1, WINDOW, 1), y_tr, epochs=200, batch_size=16, verbose=0)

    # Rolling forecast over validation window
    history = train.values.astype(np.float32).tolist()
    preds = []
    for _ in range(len(val)):
        x = np.array(history[-WINDOW:], dtype=np.float32).reshape(1, WINDOW, 1)
        yhat = model.predict(x, verbose=0)[0][0]
        preds.append(yhat)
        history.append(val.iloc[len(preds)-1])  # teacher forcing with true val

    fc_lstm = pd.Series(preds, index=val.index, name="LSTM_forecast")

    ax = train.plot(label="train", figsize=(8,4))
    val.plot(ax=ax, label="val")
    fc_lstm.plot(ax=ax, label="LSTM fc", linestyle="--")
    plt.legend(); plt.title("LSTM forecast vs validation"); plt.tight_layout()
    plt.savefig(OUT/"lstm_val.png", dpi=200); plt.show()

    def metrics(y_true, y_pred):
        mae  = np.mean(np.abs(y_true - y_pred))
        rmse = np.sqrt(np.mean((y_true - y_pred)**2))
        mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100
        return mae, rmse, mape

    mae, rmse, mape = metrics(val.values, fc_lstm.values)
    print(f"LSTM val → MAE={mae:.2f} | RMSE={rmse:.2f} | MAPE={mape:.2f}%")

except Exception as e:
    print("⚠️ TensorFlow unavailable, skipping LSTM:", e)
    fc_lstm = None


## 7) Compare forecasts (overlay) & quick summary

In [ ]:

ax = train.plot(label="train", figsize=(9,4))
val.plot(ax=ax, label="val", color="black")
if 'fc' in globals() and fc is not None:
    fc.plot(ax=ax, label="ARIMA", linestyle="--")
if 'fc_prophet' in globals() and fc_prophet is not None:
    fc_prophet.plot(ax=ax, label="Prophet", linestyle="--")
if 'fc_lstm' in globals() and fc_lstm is not None:
    fc_lstm.plot(ax=ax, label="LSTM", linestyle="--")
plt.legend(); plt.title("Validation overlay"); plt.tight_layout()
plt.savefig(OUT/"overlay.png", dpi=220); plt.show()

print("Done. Check the 'outputs/' folder for saved figures.")
